In [21]:
import requests
import pandas as pd
import re

def extract_album_id(album_link):
    match = re.search(r'/album/.*?/(\w+)\.html', album_link or "")
    return match.group(1) if match else "N/A"

def determine_album_type(track_count):
    if track_count == 1:
        return "Single"
    elif 2 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

def fetch_artist_songs(artist_name):
    url = f"http://localhost:5000/api/artistsongs?name={artist_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        songs = data.get("songs", [])
        
        records = []
        album_tracks = {}
        
        for song in songs:
            tracklist = song.get("tracklist", [])
            
            if not tracklist:
                tracklist = [{"title": song.get("title", "N/A")}]  # Nếu không có danh sách bài hát, lấy luôn tên bài hát
            
            album_id = extract_album_id(song.get("albumLink", "") or "")
            
            if album_id not in album_tracks:
                album_tracks[album_id] = len(tracklist)
            
            for track in tracklist:
                records.append({
                    "album_id": album_id,
                    "album_name": song.get("album", song.get("title", "N/A")),
                    "track_title": track["title"],
                    "release_date": song.get("releaseDate", "Unknown"),
                    "provided_by": song.get("providedBy", "Unknown"),
                    "featured_artists": song.get("featuredArtists", "Unknown"),
                    "album_owner": song.get("albumOwner", "Unknown"),
                    "linkzingmp3": song.get("link", "N/A")
                })
        
        df = pd.DataFrame(records)
        df = df.drop_duplicates(subset=["album_id", "album_name", "track_title"])
        
        # Thêm cột phân loại album
        df["album_type"] = df["album_id"].map(lambda x: determine_album_type(album_tracks.get(x, 1)))
        
        return df
    else:
        print("Error fetching data:", response.status_code)
        return None

artist_name='Obito'
df = fetch_artist_songs(artist_name)


In [22]:
df

,album_id,album_name,track_title,release_date,provided_by,featured_artists,album_owner,linkzingmp3,album_type
0,N/A,Shay Nắnggg,Shay Nắnggg,None,None,"AMEE, Obito",None,https://zingmp3.vn/bai-hat/Shay-Nanggg-AMEE-Ob...,Single
1,ZA9UAD8I,Simple Love (Single),Simple Love,28/11/2022,Warner Music Vietnam,"Obito, Seachains, Davis","Obito, Seachains",https://zingmp3.vn/bai-hat/Simple-Love-Obito-S...,Single
2,6BFZIF6B,Dám Rực Rỡ (Single),Dám Rực Rỡ,23/01/2024,Universal Music Group,"GREY D, HIEUTHUHAI, Wren Evans, Obito","GREY D, HIEUTHUHAI, Wren Evans, Obito, Kai Đinh",https://zingmp3.vn/bai-hat/Dam-Ruc-Ro-GREY-D-H...,Single
3,6B6ZCOD6,Si me you (Single),Si me you,10/10/2022,Warner Music Vietnam,Obito,Obito,https://zingmp3.vn/bai-hat/Si-me-you-Obito/Z6W...,Single
4,N/A,When You Look at Me,When You Look at Me,None,None,Obito,None,https://zingmp3.vn/bai-hat/When-You-Look-at-Me...,Single
5,6B7C9ZO0,Tiền Là Chi (Mcredit x OCEAN MOB) (Single),Tiền Là Chi (Mcredit x OCEAN MOB),22/12/2022,MIXUS,"OCEAN MOB, wAvy, Willistic, Gill, xolitxo, Obito",OCEAN MOB,https://zingmp3.vn/bai-hat/Tien-La-Chi-Mcredit...,Single
6,SC0EC0B6,OMERTÀ,BUSINESS MODE,17/05/2024,DAO Music Entertainment,"$A Milo, Gxxfy, Obito, B-Wine","$A Milo, Gxxfy",https://zingmp3.vn/bai-hat/BUSINESS-MODE-A-Mil...,Single
7,6BFOFUOD,Phong Long (Single),Phong Long,20/01/2024,Universal Music Group,"Low G, Obito, WOKEUP","Obito, Low G, WOKEUP",https://zingmp3.vn/bai-hat/Phong-Long-Low-G-Ob...,Single
8,6B0AIO0I,Soju Love (Single),Soju Love,08/03/2022,Universal Music Group,Obito,Obito,https://zingmp3.vn/bai-hat/Soju-Love-Obito/ZZ7...,Single
9,6CUCB0OO,Chỉ Cần Bạn Có Mặt (Ngày Hội Liên Quân 2024) (...,Chỉ Cần Bạn Có Mặt (Ngày Hội Liên Quân 2024),16/07/2024,TECHBEAT Music,"Obito, Wxrdie, Cheng, Garena Liên Quân Mobile","Obito, Wxrdie, Cheng, Garena Liên Quân Mobile",https://zingmp3.vn/bai-hat/Chi-Can-Ban-Co-Mat-...,Single


In [23]:
df.shape[0]

47

In [ ]:
df.to_excel('song_zingmp3.xlsx',index=False)

In [ ]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

def get_artist_tracks_all(artist_name):
    client_id = "c7e1fe3ffe674920a01f9b016e6ae5df"
    client_secret = "215c6808dea74656b3629b306182ac4b"
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
    
    # Tìm kiếm nghệ sĩ
    results = sp.search(q=artist_name, type='artist', limit=1)
    if not results['artists']['items']:
        print("Không tìm thấy nghệ sĩ.")
        return pd.DataFrame()
    
    artist_id = results['artists']['items'][0]['id']

    track_data = []
    query = f"artist:{artist_name}"
    search_results = sp.search(q=query, type='track', limit=50)  
    
    for track in search_results['tracks']['items']:
        track_title = track['name']
        link_spotify = track['external_urls']['spotify']
        album = track['album']
        album_name = album['name']
        album_release_date = album.get('release_date', 'Unknown')
        album_owner = album['artists'][0]['name']
        featured_artists = [artist['name'] for artist in track.get('artists', []) if artist['id'] != artist_id]
        featured_artists = ", ".join(featured_artists) if featured_artists else "None"

        
        # Lấy thông tin nhà cung cấp (label)
        album_id = album['id']
        album_info = sp.album(album_id)
        label = album_info.get('label', 'Unknown')

        try:
            album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
        except:
            album_release_date = "Unknown"
        
        track_data.append([
            album_name, track_title, album_release_date, 
            featured_artists if featured_artists else "None", 
            album_owner, label, link_spotify
        ])
    
    # Tạo DataFrame
    columns = ["album_name", "track_title", "release_date", "featured_artists", "album_owner", "label", "Link_Spotify"]
    df = pd.DataFrame(track_data, columns=columns)
    return df

if __name__ == "__main__":
    artist_name = 'Obito'
    df_tracks = get_artist_tracks_all(artist_name)





In [33]:
df_tracks.shape[0]

50

In [34]:
df_tracks.to_excel('{artist_name}_song_spotify.xlsx',index=False)

In [39]:
df_tracks

,album_name,track_title,release_date,featured_artists,album_owner,label,Link_Spotify
0,Lặng,1000 Ánh Mắt,27/06/2024,Shiki,Shiki,12 trái lê,https://open.spotify.com/track/5XfGQZA0ioQAWUj...
1,ii-V-I,Track 06,05/12/2024,"Tyronee, VSTRA",Tyronee,12 trái lê,https://open.spotify.com/track/72NMPuJCRsOUoXQ...
2,Đánh Đổi,Hà Nội,10/10/2023,"Shiki, VSTRA",Obito,12 trái lê,https://open.spotify.com/track/5PGh9MyKry8slG5...
3,Đánh Đổi,Đánh Đổi,10/10/2023,"Shiki, RPT MCK",Obito,12 trái lê,https://open.spotify.com/track/1UQNU5O9VttUgO1...
4,"Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...","Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",25/12/2023,"VSOUL, RPT MCK, Ronboogz, Boyzed",VSOUL,12 trái lê,https://open.spotify.com/track/4OXAneZHFJyQV1R...
5,Đánh Đổi,Tell the kids i love them,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2Asy51yFMEG8Qjh...
6,Đánh Đổi,Đầu Đường Xó Chợ,10/10/2023,"Shiki, Lăng LD",Obito,12 trái lê,https://open.spotify.com/track/2G5ujw3rrWcAXQq...
7,Đánh Đổi,Biên Giới Long Bình,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/5KXRShxe6Y54ueB...
8,Simple Love,Simple Love,28/11/2022,Seachains,Obito,12 trái lê,https://open.spotify.com/track/1hFZdk0jIkrMEri...
9,Đánh Đổi,Sài Gòn ơi,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2OucKhBwbCuIdCq...


In [ ]:
merged_df = df.merge(df_tracks, on="track_title", how="outer")
merged_df = merged_df.drop(columns=["album_name_y"])
merged_df = merged_df.rename(columns={"album_id":"Mã định danh album ZingMP3","album_name_x": "album_name","track_title":"tracklist(danh sách bài hát)","release_date_x": "Ngày phát hành trên ZingMP3","featured_artists_x":"Song artist(nghệ sĩ tham gia bài hát)","provided_by":"Cung cấp bởi","album_owner_x":'Album artist (nghệ sĩ sở hữu album)*',"linkzingmp3":"Link_ZingMP3",           	
"release_date_y": "Ngày phát hành trên Spotify","featured_artists_y":"Song artist(nghệ sĩ tham gia bài hát) trên S","label":"Cung cấp bởi","album_owner_y":'Album artist (nghệ sĩ sở hữu album)*'})
# desired_columns = [
#  "album_id","album_name","Album artist (nghệ sĩ sở hữu album)*","album_type","tracklist(danh sách bài hát)","Ngày phát hành trên ZingMP3","Cung cấp bởi","Song artist(nghệ sĩ tham gia bài hát)","Ngày phát hành trên Spotify", "Song artist(nghệ sĩ tham gia bài hát)","Album artist (nghệ sĩ sở hữu album)*","Cung cấp bởi","Mã định danh album ZingMP3","Link_ZingMP3","Link_Spotify"]
# merged_df = merged_df[desired_columns]
merged_df

,album_id,album_name,tracklist(danh sách bài hát),Ngày phát hành trên ZingMP3,Cung cấp bởi,Song artist(nghệ sĩ tham gia bài hát),Album artist (nghệ sĩ sở hữu album)*,Link_ZingMP3,album_type,Ngày phát hành trên Spotify,Song artist(nghệ sĩ tham gia bài hát),Album artist (nghệ sĩ sở hữu album)*,Cung cấp bởi,Link_Spotify
0,6CZ7U60W,Lặng,1000 Ánh Mắt,27/06/2024,Warner Music Vietnam,"Shiki, Obito",Shiki,https://zingmp3.vn/bai-hat/1000-Anh-Mat-Shiki-...,EP,27/06/2024,Shiki,Shiki,12 trái lê,https://open.spotify.com/track/5XfGQZA0ioQAWUj...
1,6BDOI9WU,Đánh Đổi,16,10/10/2023,Warner Music Vietnam,"Obito, Shiki, Lăng LD","Obito, Shiki",https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,Regular,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/3jwzesnDs0jY7Ke...
2,SC0EC0B6,OMERTÀ,BUSINESS MODE,17/05/2024,DAO Music Entertainment,"$A Milo, Gxxfy, Obito, B-Wine","$A Milo, Gxxfy",https://zingmp3.vn/bai-hat/BUSINESS-MODE-A-Mil...,Single,18/05/2024,"$A Milo, B-Wine, Gxxfy",$A Milo,MaiDao Music,https://open.spotify.com/track/1nLkB5tZaxAIbJ7...
3,6BDOI9WU,Đánh Đổi,Backstage Freestyle,10/10/2023,Warner Music Vietnam,"Obito, Shiki, Lăng LD","Obito, Shiki",https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,Regular,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/6Kvm6QRtOOM6bMe...
4,6BDOI9WU,Đánh Đổi,Biên Giới Long Bình,10/10/2023,Warner Music Vietnam,"Obito, Shiki, Lăng LD","Obito, Shiki",https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,Regular,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/5KXRShxe6Y54ueB...
5,N/A,Boom,Boom,None,None,"Lope Phạm, Obito",None,https://zingmp3.vn/bai-hat/Boom-Lope-Pham-Obit...,Single,NaN,NaN,NaN,NaN,NaN
6,6BEEZ96E,"Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...","Buồn Hay Vui (feat. RPT MCK, Obito, Ronboogz &...",25/12/2023,Warner Music Vietnam,"VSOUL, MCK, Obito, Ronboogz, Boyzed","VSOUL, MCK, Obito, Ronboogz, Boyzed",https://zingmp3.vn/bai-hat/Buon-Hay-Vui-feat-R...,Single,25/12/2023,"VSOUL, RPT MCK, Ronboogz, Boyzed",VSOUL,12 trái lê,https://open.spotify.com/track/4OXAneZHFJyQV1R...
7,6BDOI9WU,Đánh Đổi,CL5 (interlude),10/10/2023,Warner Music Vietnam,"Obito, Shiki, Lăng LD","Obito, Shiki",https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,Regular,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2pAwijeWeUsyoXF...
8,6BW0ICWD,Can We Love (Single),Can We Love,24/03/2022,Universal Music Group,Obito,Obito,https://zingmp3.vn/bai-hat/Can-We-Love-Obito/Z...,Single,24/03/2022,None,Obito,Universal Music Indochina,https://open.spotify.com/track/7iXHkRf9Z4x6ona...
9,6BDOI9WU,Đánh Đổi,Champion,10/10/2023,Warner Music Vietnam,"Obito, Shiki, Lăng LD","Obito, Shiki",https://zingmp3.vn/bai-hat/Dau-Duong-Xo-Cho-Ob...,Regular,10/10/2023,Shiki,Obito,12 trái lê,https://open.spotify.com/track/2rG6N3Czmz552Wm...


In [19]:
print("Columns in df_spotify:", df_spotify.columns)
print("Columns in df_zingmp3:", df_zingmp3.columns)


NameError: name 'df_spotify' is not defined

In [ ]:
def preprocess_dataframe(df):
    print("Available columns before renaming:", df.columns)  # Kiểm tra cột trước khi đổi tên
    
    # Đổi tên cột về chuẩn (viết thường, không có khoảng trắng)
    column_mapping = {
        "Track Title": "track_title",
        "Release Date": "release_date",
        "Zing Release Date": "release_date",
        "Zing Featured Artists": "featured_artists",
        "Featured Artists": "featured_artists",
        "Zing Album Owner": "album_owner",
        "Album Owner": "album_owner",
        "Zing MP3 Link": "linkzingmp3",
        "Spotify Link": "spotify_link",
        "Album ID": "album_id"
    }
    df = df.rename(columns=column_mapping)

    print("Available columns after renaming:", df.columns)  # Kiểm tra sau khi đổi tên

    # Kiểm tra xem các cột quan trọng có tồn tại không
    required_columns = ["track_title", "release_date", "album_id", "album_name"]
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        raise ValueError(f"Missing required columns: {missing_columns}")

    # Chuẩn hóa dữ liệu
    df["track_title"] = df["track_title"].astype(str).apply(normalize_song_title)
    df["release_date"] = df["release_date"].astype(str).apply(normalize_date)
    
    # Loại bỏ trùng lặp dựa trên (album_id, album_name, track_title)
    df = df.drop_duplicates(subset=["album_id", "album_name", "track_title"])
    
    return df


In [ ]:
import requests
import pandas as pd
import re
import unicodedata

def normalize_title(title):
    """Chuẩn hóa tên bài hát: bỏ dấu, ký tự đặc biệt, viết thường, xóa khoảng trắng thừa"""
    title = title.lower()  # Chuyển về chữ thường
    title = unicodedata.normalize('NFKD', title).encode('ascii', 'ignore').decode('utf-8')  # Loại bỏ dấu tiếng Việt
    title = re.sub(r'[^a-z0-9\s]', '', title)  # Xóa ký tự đặc biệt, chỉ giữ chữ cái và số
    title = re.sub(r'\s+', ' ', title).strip()  # Xóa khoảng trắng thừa
    return title

def extract_album_id(album_link):
    match = re.search(r'/album/.*?/(\w+)\.html', album_link or "")
    return match.group(1) if match else "N/A"

def fetch_artist_songs(artist_name):
    url = f"http://localhost:5000/api/artistsongs?name={artist_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        songs = data.get("songs", [])

        records = []
        for song in songs:
            tracklist = song.get("tracklist", [])
            
            if not tracklist:
                tracklist = [{"title": song.get("title", "N/A")}]  # Nếu không có danh sách bài hát, lấy luôn tên bài hát
            
            album_id = extract_album_id(song.get("albumLink", "") or "")
            
            for track in tracklist:
                records.append({
                    "album_id": album_id,
                    "album_name": song.get("album", song.get("title", "N/A")),
                    "track_title": normalize_title(track["title"]),  # Chuẩn hóa tên bài hát
                    "release_date": song.get("releaseDate", "Unknown"),
                    "provided_by": song.get("providedBy", "Unknown"),
                    "featured_artists": song.get("featuredArtists", "Unknown"),
                    "album_owner": song.get("albumOwner", "Unknown"),
                    "linkzingmp3": song.get("link", "N/A")
                })
        
        df = pd.DataFrame(records)
        df = df.drop_duplicates(subset=["album_id", "album_name", "track_title"])
        return df
    else:
        print("Error fetching data:", response.status_code)
        return None

df = fetch_artist_songs("Obito")

